In [1]:
import gym
import numpy as np
rm='Pong-v4'

In [2]:
%matplotlib inline
from matplotlib import animation
import matplotlib.pyplot as plt
from IPython.display import display, HTML

def display_frames_as_gif(frames):
    plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 144)
    patch = plt.imshow(frames[0])
    plt.axis('off')
    def animate(i):
        patch.set_data(frames[i])
    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    plt.close(anim._fig)
    display(HTML(anim.to_jshtml()))

In [3]:
gym.new_step_api=True
env = gym.make(rm)
# model initialization
H = 800 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
#learning_rate = 1e-3
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  I[I == 144] = 0 # erase background (background type 1)
  I[I == 109] = 0 # erase background (background type 2)
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  """ take 1D float array of rewards and compute discounted reward """
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):
  # preprocess the observation, set input to network to be difference image
  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
    # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

    # if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
    #   print('ep {}: game finished, reward: {}'.format(episode_number, reward) + ('' if reward == -1 else ' !!!!!!!!'))

C:\Users\Sagardeep Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\Sagardeep Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
%time hist1 = train_model(env, model, total_episodes=6000)

C:\Users\Sagardeep Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -21.000000. running mean: -21.000000
episode 2.000000, reward total was -21.000000. running mean: -21.000000
episode 3.000000, reward total was -20.000000. running mean: -20.990000
episode 4.000000, reward total was -20.000000. running mean: -20.980100
episode 5.000000, reward total was -20.000000. running mean: -20.970299
episode 6.000000, reward total was -20.000000. running mean: -20.960596
episode 7.000000, reward total was -21.000000. running mean: -20.960990
episode 8.000000, reward total was -21.000000. running mean: -20.961380
episode 9.000000, reward total was -21.000000. running mean: -20.961766
episode 10.000000, reward total was -21.000000. running mean: -20.962149
episode 11.000000, reward total was -20.000000. running mean: -20.952527
episode 12.000000, reward total was -21.000000. running mean: -20.953002
episode 13.000000, reward total was -20.000000. running mean: -20.943472
episode 14.000000, reward total was -20.000000. running mean

episode 114.000000, reward total was -21.000000. running mean: -20.650190
episode 115.000000, reward total was -20.000000. running mean: -20.643688
episode 116.000000, reward total was -21.000000. running mean: -20.647251
episode 117.000000, reward total was -21.000000. running mean: -20.650779
episode 118.000000, reward total was -21.000000. running mean: -20.654271
episode 119.000000, reward total was -20.000000. running mean: -20.647728
episode 120.000000, reward total was -20.000000. running mean: -20.641251
episode 121.000000, reward total was -20.000000. running mean: -20.634838
episode 122.000000, reward total was -20.000000. running mean: -20.628490
episode 123.000000, reward total was -21.000000. running mean: -20.632205
episode 124.000000, reward total was -21.000000. running mean: -20.635883
episode 125.000000, reward total was -19.000000. running mean: -20.619524
episode 126.000000, reward total was -20.000000. running mean: -20.613329
episode 127.000000, reward total was -

episode 225.000000, reward total was -21.000000. running mean: -20.444428
episode 226.000000, reward total was -21.000000. running mean: -20.449984
episode 227.000000, reward total was -21.000000. running mean: -20.455484
episode 228.000000, reward total was -20.000000. running mean: -20.450929
episode 229.000000, reward total was -20.000000. running mean: -20.446420
episode 230.000000, reward total was -21.000000. running mean: -20.451955
episode 231.000000, reward total was -21.000000. running mean: -20.457436
episode 232.000000, reward total was -20.000000. running mean: -20.452862
episode 233.000000, reward total was -20.000000. running mean: -20.448333
episode 234.000000, reward total was -21.000000. running mean: -20.453850
episode 235.000000, reward total was -19.000000. running mean: -20.439311
episode 236.000000, reward total was -20.000000. running mean: -20.434918
episode 237.000000, reward total was -19.000000. running mean: -20.420569
episode 238.000000, reward total was -

episode 336.000000, reward total was -20.000000. running mean: -20.340075
episode 337.000000, reward total was -20.000000. running mean: -20.336674
episode 338.000000, reward total was -20.000000. running mean: -20.333307
episode 339.000000, reward total was -21.000000. running mean: -20.339974
episode 340.000000, reward total was -21.000000. running mean: -20.346574
episode 341.000000, reward total was -21.000000. running mean: -20.353109
episode 342.000000, reward total was -20.000000. running mean: -20.349577
episode 343.000000, reward total was -21.000000. running mean: -20.356082
episode 344.000000, reward total was -19.000000. running mean: -20.342521
episode 345.000000, reward total was -21.000000. running mean: -20.349096
episode 346.000000, reward total was -21.000000. running mean: -20.355605
episode 347.000000, reward total was -21.000000. running mean: -20.362049
episode 348.000000, reward total was -21.000000. running mean: -20.368428
episode 349.000000, reward total was -

...

episode 4632.000000, reward total was -17.000000. running mean: -17.007743
episode 4633.000000, reward total was -15.000000. running mean: -16.987666
episode 4634.000000, reward total was -21.000000. running mean: -17.027789
episode 4635.000000, reward total was -16.000000. running mean: -17.017511
episode 4636.000000, reward total was -17.000000. running mean: -17.017336
episode 4637.000000, reward total was -19.000000. running mean: -17.037163
episode 4638.000000, reward total was -18.000000. running mean: -17.046791
episode 4639.000000, reward total was -19.000000. running mean: -17.066323
episode 4640.000000, reward total was -18.000000. running mean: -17.075660
episode 4641.000000, reward total was -18.000000. running mean: -17.084904
episode 4642.000000, reward total was -16.000000. running mean: -17.074055
episode 4643.000000, reward total was -17.000000. running mean: -17.073314
episode 4644.000000, reward total was -19.000000. running mean: -17.092581
episode 4645.000000, rewa

episode 4742.000000, reward total was -18.000000. running mean: -17.160002
episode 4743.000000, reward total was -16.000000. running mean: -17.148402
episode 4744.000000, reward total was -19.000000. running mean: -17.166918
episode 4745.000000, reward total was -17.000000. running mean: -17.165249
episode 4746.000000, reward total was -15.000000. running mean: -17.143596
episode 4747.000000, reward total was -14.000000. running mean: -17.112160
episode 4748.000000, reward total was -18.000000. running mean: -17.121039
episode 4749.000000, reward total was -17.000000. running mean: -17.119828
episode 4750.000000, reward total was -13.000000. running mean: -17.078630
episode 4751.000000, reward total was -13.000000. running mean: -17.037844
episode 4752.000000, reward total was -17.000000. running mean: -17.037465
episode 4753.000000, reward total was -14.000000. running mean: -17.007091
episode 4754.000000, reward total was -16.000000. running mean: -16.997020
episode 4755.000000, rewa

episode 4852.000000, reward total was -13.000000. running mean: -16.899086
episode 4853.000000, reward total was -20.000000. running mean: -16.930095
episode 4854.000000, reward total was -18.000000. running mean: -16.940794
episode 4855.000000, reward total was -17.000000. running mean: -16.941386
episode 4856.000000, reward total was -17.000000. running mean: -16.941972
episode 4857.000000, reward total was -20.000000. running mean: -16.972552
episode 4858.000000, reward total was -13.000000. running mean: -16.932827
episode 4859.000000, reward total was -18.000000. running mean: -16.943499
episode 4860.000000, reward total was -18.000000. running mean: -16.954064
episode 4861.000000, reward total was -14.000000. running mean: -16.924523
episode 4862.000000, reward total was -18.000000. running mean: -16.935278
episode 4863.000000, reward total was -17.000000. running mean: -16.935925
episode 4864.000000, reward total was -14.000000. running mean: -16.906566
episode 4865.000000, rewa

episode 4962.000000, reward total was -21.000000. running mean: -16.964177
episode 4963.000000, reward total was -17.000000. running mean: -16.964536
episode 4964.000000, reward total was -13.000000. running mean: -16.924890
episode 4965.000000, reward total was -19.000000. running mean: -16.945641
episode 4966.000000, reward total was -16.000000. running mean: -16.936185
episode 4967.000000, reward total was -17.000000. running mean: -16.936823
episode 4968.000000, reward total was -16.000000. running mean: -16.927455
episode 4969.000000, reward total was -14.000000. running mean: -16.898180
episode 4970.000000, reward total was -19.000000. running mean: -16.919198
episode 4971.000000, reward total was -15.000000. running mean: -16.900006
episode 4972.000000, reward total was -18.000000. running mean: -16.911006
episode 4973.000000, reward total was -17.000000. running mean: -16.911896
episode 4974.000000, reward total was -18.000000. running mean: -16.922777
episode 4975.000000, rewa

episode 5072.000000, reward total was -15.000000. running mean: -16.790737
episode 5073.000000, reward total was -16.000000. running mean: -16.782829
episode 5074.000000, reward total was -14.000000. running mean: -16.755001
episode 5075.000000, reward total was -11.000000. running mean: -16.697451
episode 5076.000000, reward total was -17.000000. running mean: -16.700477
episode 5077.000000, reward total was -16.000000. running mean: -16.693472
episode 5078.000000, reward total was -18.000000. running mean: -16.706537
episode 5079.000000, reward total was -19.000000. running mean: -16.729472
episode 5080.000000, reward total was -17.000000. running mean: -16.732177
